# Çözüm

2010LP33 asteroidi için $a = 2.55$ $AB$, $e=0,42$, $\tau=2458344$, 2343327492 gün - 13 Ağustos 2018 17:37:28, $P=1491.04$ gün veriliyor. Yörüngesi tanımlı asteroidin enberi geçişinden bu yana heliosentrik uzaklığını ve gerçek anomalisini ($\nu$) bulunuz.

Heliosentrik uzaklık, odaklarının birinde Güneş'in bulunduğu eliptik yörünge takip eden bir asteroidin $r$ uzaklığı
$$ r = \frac{h^2}{\mu}\frac{1}{1 + ecos\nu}$$
bulunmalıdır.

bilinmeyenler, $h$ ve $\nu$.

## Önemli Formüller

$$ r = a(1-ecosE) $$
$$ \tan{\frac{\nu}{2}} = \sqrt{\frac{1 + e}{1 - e}} \tan{\frac{E}{2}} $$
$$ M = E - esinE = \frac{2\pi}{P}(t - \tau) $$
$$ P^2 = \frac{4\pi^2a^3}{\mu}$$

$\mu_{sun} = 1.327 \times 10^{11} \frac{km^3}{s^2}$

Çözüme giden adımlar:

1. Ortalama anomaliyi $M_e$ bul.
2. Kepler denklemini ($E - esinE$) çöz ve dış anomaliyi ($E$) bul. Analitik çözümü bulunmayan bu denklemin çözümünde Newton-Raphson nümerik yöntemi kullanılacak.
3. Tanjantlı fonksiyonda yerine koyarak gerçek anomaliyi $\nu$ bul.

In [1]:
import numpy as np
import pandas as pd

In [2]:
a = 2.55        # yarı-büyük eksen [AB]
e = 0.42        # dışmerkezlik
tau = 2_458_344 # enberiden geçme zamanı
p = 1491.04     # yörünge dönemi (gün)
t = 2_343_327_492  # gün

$P=1491.04$ gün verildiği için önce bu değeri ve $t$'yi $[sn]$ birimine, $a$ da astronomik birim uzaklığında verildiği için bunu da $[km]$ birimine çevirelim.

In [3]:
p = p * 24 * 60 * 60
t = t * 24 * 60 * 60
a = 2.55 * 149_597_871 # 1 AB = 149,597,871 km

In [4]:
print(p)
print(a)

128825856.0
381474571.04999995


In [5]:
def get_mean_anomaly(t, tau=tau):
    return (2 * np.pi / p) * (t - tau)

In [6]:
mean_anomaly = get_mean_anomaly(t)
mean_anomaly

9874691.9525087

Ortalama anomali $M_e$ teoride, $[0, 2\pi]$ aralığında olur, fakat $t$ değeri çok büyükse $M_e$ de büyük çıkabilir. Haliyle sonucu uygun aralıkta normalize etmek gerekecektir.

In [7]:
normalized_mean_anomaly = mean_anomaly % (2 * np.pi)
mean_anomaly = normalized_mean_anomaly

In [8]:
mean_anomaly

0.22463341834917117

Burada $M_e$ radyan biriminde
$$ M_e = 0.22463341834917117$$

## Newton-Raphson Metodu

Newton-Rapshon metodu, kısaca $f(E) = E - esinE$ ve $f'(E) = 1 - ecosE$ olmak üzere

$$ M < \pi, E = M + \frac{e}{2}$$
$$ M > \pi, E = M - \frac{e}{2}$$

başlangıç tahminleri ile verilen $E_0$ değerinden başlayarak

$$ E_{i+1} = E_i - \frac{E_i-esinE_i-M}{1-ecosE_i}$$

iterasyonunu gerçekleştiren yönteme denir. Burada

$$ E_{i+1} = E_i - \frac{f(E_i)}{f'(E_i)} $$

kontrolü yapılarak

$$ \lvert \frac{f(x_i)}{f'(x_i)} \rvert$$

ifadesi, $10^{-6}$ sayısından büyük ya da küçük olana kadar iterasyon yapılır.

$$ M > \pi, E_0 = M - \frac{e}{2} $$
olsun. O halde

In [9]:
e0 = mean_anomaly - (e / 2)
tolerance_value = 1e-6

In [10]:
def iterate_newton(element, tolerance=tolerance_value, max_iteration_threshold=100):
    i = 1
    while (i <= max_iteration_threshold):
        f = element - (e * np.sin(element)) - mean_anomaly
        f_derivative = 1 - (e * np.cos(element))
        control = f / f_derivative
        
        print(f"{i:2d}. step; E: {element:10.6f}; control: {np.abs(control):10.6e}")
        
        element = element - control
        if np.abs(control) < tolerance:
            break
        i += 1
    return element

Yukarıdaki hücrede `control` adlı değişken değişim miktarını belirten $ \frac{f(E_i)}{f'(E_i)} $ oranını verir. Bu değişken küçüldükçe her iterasyonda değişim miktarı da azalır (çözüme yakınsar).

In [11]:
excentric_anomaly = iterate_newton(e0)

 1. step; E:   0.014633; control: 3.726363e-01
 2. step; E:   0.387270; control: 6.575604e-03
 3. step; E:   0.380694; control: 5.590785e-06
 4. step; E:   0.380689; control: 3.997812e-12


In [12]:
excentric_anomaly

np.float64(0.38068853361942634)

$E$'den $\nu$'ye geçiş ise
$$ E = 2\arctan({\sqrt{\frac{1-e}{1+e}}\tan{\frac{\nu}{2}}}) $$
$$ \nu = 2\arctan({\sqrt{\frac{1+e}{1-e}}\tan{\frac{E}{2}}}) $$
bağıntıları yardımıyla yapılır.

In [13]:
square_root = np.sqrt((1+e) / (1-e))
true_anomaly = 2 * np.arctan(square_root * np.tan(excentric_anomaly / 2))

In [14]:
true_anomaly

np.float64(0.5856296459366132)

In [15]:
true_anomaly_deg = np.rad2deg(true_anomaly)
true_anomaly_deg

np.float64(33.55410706990865)

Sonuç olarak
$$ \nu = 33.55^\circ $$

In [16]:
np.rad2deg(0.22)

np.float64(12.60507149287811)

## Heliosentrik Uzaklık

$$ r = a(1 - cosE) $$

In [17]:
def get_heliocentric_distance(excentric_anomaly):
    return a * (1 - np.cos(excentric_anomaly))

In [18]:
get_heliocentric_distance(excentric_anomaly)

np.float64(27310136.76399116)